In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Deliverable 2

Step 2. Extract the dataset you used in Deliverable 1.

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

Step 3. Recreate the `vine_table`, and perform your analysis using the steps.

In [4]:
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

1. Filter the data and create a new DataFrame or table to retrieve all the rows where the `total_votes` count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [5]:
over_20_total_votes_df = vine_df.filter(vine_df['total_votes'] > 20)
over_20_total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|     

2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of `helpful_votes` divided by `total_votes` is equal to or greater than 50%.

In [6]:
helpful_voting_df = over_20_total_votes_df.filter((over_20_total_votes_df['helpful_votes'] / over_20_total_votes_df['total_votes']) >= 0.5)
helpful_voting_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
|R1BM9RBQWI62O2|          5|           43|         60|   N|     

3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), `vine == 'Y'`.

In [7]:
vine_program_yes_df = helpful_voting_df.filter(helpful_voting_df['vine'] == 'Y')
vine_program_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|
|R3J8OI5CB74P5K|          1|           22|         25|   Y|                N|
| RH39LMKN6AZDC|          5|           33|         40|   Y|                N|
| R8RD8K0ESJSRD|          5|           21|         21|   Y|                N|
| R6P04SW0Z1FCR|          5|           20|         25|   Y|     

4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), `vine == 'N'`.

In [8]:
vine_program_no_df = helpful_voting_df.filter(helpful_voting_df['vine'] == 'N')
vine_program_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
|R1BM9RBQWI62O2|          5|           43|         60|   N|     

5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).


In [9]:
# Calculate total reviews for Vine participants
total_reviews_vine_yes = vine_program_yes_df.count()

# Calculate total 5-star reviews for Vine participants
total_5_star_reviews_vine_yes = vine_program_yes_df.filter(vine_program_yes_df['star_rating'] == '5').count()

# Calculate percentage of 5-star to total reviews for Vine participants
five_star_percentage_vine_yes = round((total_5_star_reviews_vine_yes / total_reviews_vine_yes)*100,2)

print(f'Total Reviews (Vine Participants): {total_reviews_vine_yes}')
print(f'Total 5-Star Reviews (Vine Participants): {total_5_star_reviews_vine_yes}')
print(f'5-Star Review percentage (Vine Participants): {five_star_percentage_vine_yes}%')

Total Reviews (Vine Participants): 373
Total 5-Star Reviews (Vine Participants): 171
5-Star Review percentage (Vine Participants): 45.84%


In [10]:
# Calculate total reviews for non-Vine participants
total_reviews_vine_no = vine_program_no_df.count()

# Calculate total 5-star reviews for non-Vine participants
total_5_star_reviews_vine_no = vine_program_no_df.filter(vine_program_no_df['star_rating'] == '5').count()

# Calculate percentage of 5-star to total reviews for non-Vine participants
five_star_percentage_vine_no = round((total_5_star_reviews_vine_no / total_reviews_vine_no)*100,2)

print(f'Total Reviews (non-Vine Participants): {total_reviews_vine_no}')
print(f'Total 5-Star Reviews (non-Vine Participants): {total_5_star_reviews_vine_no}')
print(f'5-Star Review percentage (non-Vine Participants): {five_star_percentage_vine_no}%')

Total Reviews (non-Vine Participants): 45706
Total 5-Star Reviews (non-Vine Participants): 22546
5-Star Review percentage (non-Vine Participants): 49.33%


## Additional Study removing non-verfied purchasers from non-Vine participants
Vine participants do not purchase the product themselves, therefore would be mostly filtered out if applied to their dataset.

In [11]:
verified_helpful_voting_df = helpful_voting_df.filter(helpful_voting_df['verified_purchase'] == 'Y')
verified_helpful_voting_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
| RWB6V9IYCOJFX|          4|           39|         41|   N|                Y|
|R15FA7GCXKC7YE|          1|           40|         41|   N|                Y|
|R2OQAI6AV8AH8Z|          1|           19|         32|   N|                Y|
|R2XD5CWZ4Z87ZR|          5|           19|         25|   N|     

In [15]:
verified_vine_program_no_df = verified_helpful_voting_df.filter(verified_helpful_voting_df['vine'] == 'N')
verified_vine_program_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
| RWB6V9IYCOJFX|          4|           39|         41|   N|                Y|
|R15FA7GCXKC7YE|          1|           40|         41|   N|                Y|
|R2OQAI6AV8AH8Z|          1|           19|         32|   N|                Y|
|R2XD5CWZ4Z87ZR|          5|           19|         25|   N|     

In [18]:
# Calculate total reviews for non-Vine participants
verified_total_reviews_vine_no = verified_vine_program_no_df.count()

# Calculate total 5-star reviews for non-Vine participants
verified_total_5_star_reviews_vine_no = verified_vine_program_no_df.filter(verified_vine_program_no_df['star_rating'] == '5').count()

# Calculate percentage of 5-star to total reviews for non-Vine participants
verified_five_star_percentage_vine_no = round((verified_total_5_star_reviews_vine_no / verified_total_reviews_vine_no)*100,2)

print(f'Total Verified Reviews (non-Vine Participants): {verified_total_reviews_vine_no}')
print(f'Total 5-Star Verified Reviews (non-Vine Participants): {verified_total_5_star_reviews_vine_no}')
print(f'5-Star Review Verified percentage (non-Vine Participants): {verified_five_star_percentage_vine_no}%')

Total Verified Reviews (non-Vine Participants): 30577
Total 5-Star Verified Reviews (non-Vine Participants): 15908
5-Star Review Verified percentage (non-Vine Participants): 52.03%
